In [10]:
import xml.etree.ElementTree as ET
from os import listdir, mkdir
from os.path import join

In [11]:
filepath = join('dataset', 'ECFR-title14.xml') # file already fetched from https://www.govinfo.gov/bulkdata/ECFR/title-14/ECFR-title14.xml
tree = ET.parse(filepath)
root = tree.getroot()

In [12]:
if 'temp' not in listdir():
    mkdir('temp')

filepath = join('temp', 'ECFR-title14.txt')

In [13]:
with open(filepath, 'w', encoding='utf-8') as file:
    for element in root.iter():
        if element.tag == 'DIV1':
            if element.attrib['N'] == '3':
                break
        [file.write(line) for line in element.itertext() if line[0] != '\n']
        #[print(line) for line in element.itertext() if line[0] != '\n']

### Trash

In [5]:
os.listdir()

NameError: name 'os' is not defined

In [ ]:
'temp' in os.listdir()